# Representations of Color
Converting input Hex codes to RGB and HSL.

In [ ]:
import pandas as pd
from pycolorkit import ColorConverter, ColorGenerator

In [125]:
# Valid entries
df = pd.DataFrame(['#FF0000', '#00ff00', '#fff'], columns=['hex'])
df.head()

,hex
0,#FF0000
1,#00ff00
2,#fff


## Hex to RGB
Convert every set of two hexidecimals to decimals.

In [ ]:
def format_hex(s:str):

    s = s.replace('#','')
    
    if len(s) == 3:
        s = [s[i] * 2 for i in range(3)] 


    return ''.join(s)

In [ ]:
df['hex'] = df['hex'].apply(format_hex)
df

,hex
0,FF0000
1,00ff00
2,ffffff


In [ ]:
def hex_to_rgb(h:str):
    # hexadecimal to decimal
    return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))

In [129]:
df['rgb'] = df['hex'].apply(hexToRGB)
df

,hex,rgb
0,FF0000,"(255, 0, 0)"
1,00ff00,"(0, 255, 0)"
2,ffffff,"(255, 255, 255)"


## RGB to HSL
This human-friendly conversion allows us to represent color on a color wheel by determining where the color point exists in relation to pure red, green, or blue. We can do this in 4 steps:

**Step 1**. Calculate the max, min, and chroma.  

$
M = max(R, G, B) \\
m = min(R, G, B) \\
C = range(R, G, B) = M - m$

**Step 2**. Calculate the hue measured in degrees `[0°, 360°)`.  

$H' = 
\begin{cases}
\text{undefined}, & \text{if } C = 0 \\
\frac{G-B}{C} \bmod 6, & \text{if } M = R \\
\frac{B-R}{C} + 2, & \text{if } M = G \\
\frac{R-G}{C} + 4, & \text{if } M = B
\end{cases}$

Then, $H = 60° × H'$

**Step 3**. Calculate Lightness measuring color intensity using set `[0%, 100%)`.

$L = mid(R, G, B) = \frac{1}{2}(M + m)$
    

**Step 4**. Calculate Saturation determined by the amount of grayscale or pure color from `[0%, 100%)`.

$S_L =
\begin{cases}
0, & \text{if } L = 1 \text{ or } L = 0 \\
\frac{C}{1-|2L-1|}, & \text{otherwise}
\end{cases}$
  

See [wiki page](https://en.wikipedia.org/wiki/HSL_and_HSV).

In [130]:
def rgbToHSL(rgb):
    # Change range from 0-255 to 0-100%
    r = rgb[0] / 255
    g = rgb[1] / 255
    b = rgb[2] / 255

    # Determine Chroma component values 
    cmax = max(r, g, b)
    cmin = min(r, g, b)
    delta = cmax - cmin

    # Calculate Hue: Distance from white (0-350 degrees)
    if delta == 0:
        h = 0
    elif cmax == r:                # more red
        h = (g - b) / delta % 6
    elif cmax == g:
        h = (b - r) / delta + 2.0   # more green
    else:                       # more blue
        h = (r - g) / delta + 4.0

    # Calulate Lightness: Midrange of RGB (0-100%)
    l = (cmax + cmin) / 2

    # Calulate Saturation: Gray to Pure (0-100%)
    if delta == 0:
        s = 0
    else:
        s = delta / (1 - abs(2 * l - 1))

    h = round(h * 60)
    s = round(s * 100)
    l = round(l * 100)
    return (h, s, l)

In [131]:
df['hsl'] = df['rgb'].apply(rgbToHSL)
df

,hex,rgb,hsl
0,FF0000,"(255, 0, 0)","(0, 100, 50)"
1,00ff00,"(0, 255, 0)","(120, 100, 50)"
2,ffffff,"(255, 255, 255)","(0, 0, 100)"


# Lightness Adjustments
Center the base lightness value and generate a range of shades and tints in the same hue. Tints add light to the lightness (`l`). Shades removes light from the lightness (`l`). Functions assume that l is a decimal value. The base color value is included in the final output.

In [132]:
# Fixed lightness values from 10% to 90%
def sequence(hsl, n=10, min_l=10, max_l=90):
    h = round(hsl[0])
    s = round(hsl[1])
    l = round(hsl[2])
    
    shade_diff = int((l - min_l) / (n/2))
    shades = [i for i in range(min_l, l, shade_diff)]

    tint_diff = int((max_l - l) / (n/2))
    tints = [i for i in range(l, max_l, tint_diff)]

    return [(h,s,l) for l in shades+tints]

In [133]:
df['sequential_hsl'] = df['hsl'].apply(sequence)
df[['hsl','sequential_hsl']]

,hsl,sequential_hsl
0,"(0, 100, 50)","[(0, 100, 10), (0, 100, 18), (0, 100, 26), (0,..."
1,"(120, 100, 50)","[(120, 100, 10), (120, 100, 18), (120, 100, 26..."
2,"(0, 0, 100)","[(0, 0, 10), (0, 0, 28), (0, 0, 46), (0, 0, 64..."


# Complimentary Colors
Colors compliment each other when they are directly opposite one another on the color wheel. Add $180\degree$ to the base color hue value.

In [134]:
def compliment(hsl):
    h = hsl[0]
    s = hsl[1]
    l = hsl[2]

    if (h + 180) > 360:
        return (h-180, s, l)
    else:
        return (h+180, s, l)

In [135]:
df['compliment_hsl'] = df['hsl'].apply(compliment)
df[['hsl','compliment_hsl']]

,hsl,compliment_hsl
0,"(0, 100, 50)","(180, 100, 50)"
1,"(120, 100, 50)","(300, 100, 50)"
2,"(0, 0, 100)","(180, 0, 100)"


You can also create a sequence of colors between the base colors and it's complimentary colors since they have th same lightness values.

In [136]:
# sequence lightness for complimentary colors
df['diverging_hsl'] = df['compliment_hsl'].apply(sequence)

# reverse list order
df['diverging_hsl'] = df['diverging_hsl'].apply(lambda x: x[::-1])

# combine with base color sequence
df['diverging_hsl'] = df['diverging_hsl'] + df['sequential_hsl']

# preview colors
print([i for i in df[['diverging_hsl']].iloc[0]])

[[(180, 100, 82), (180, 100, 74), (180, 100, 66), (180, 100, 58), (180, 100, 50), (180, 100, 42), (180, 100, 34), (180, 100, 26), (180, 100, 18), (180, 100, 10), (0, 100, 10), (0, 100, 18), (0, 100, 26), (0, 100, 34), (0, 100, 42), (0, 100, 50), (0, 100, 58), (0, 100, 66), (0, 100, 74), (0, 100, 82)]]


# HSL to Hex
Returning generated colors to machine compatible color codes.

### HSL to RGB
Step 1. Find chroma.

$C = (1 - |2L - 1|) * S_L$
  

Step 2. Find a point (R,G,B) with the same hue and chroma.

$H' = \frac{H}{60\degree}$
    
$X = C s (1 |H' \mod 2 - 1|) \\$  
 
$(R_1, G_1, B_1) = 
\begin{cases}
(C, X, 0) & \text{if } 0<=H'<1 \\
(X, C, 0) & \text{if } 1<=H'<2 \\
(0, C, X) & \text{if } 2<=H'<3 \\
(0, X, C) & \text{if } 3<=H'<4 \\
(X, 0, C) & \text{if } 4<=H'<5 \\
(C, 0, X) & \text{if } 5<=H'<6 \\
\end{cases}$

Step 3. Add amount to match the lightness.

$m = L - \frac{C}{2}$  
 
$(R, G, B) = (R_1 + m, G_1 + m, B_1 + m)$

In [137]:
def hslToRGB(colors:list):
    h = colors[0]
    s = colors[1]/100
    l = colors[2]/100

    # Find chroma
    chroma = (1 - abs(2*l - 1)) * s

    # Find point (r,g,b) of RGB cube
    X = chroma * (1 - abs((h/60) % 2 - 1))

    if      0 <= h < 1: rgb = [chroma, X, 0]
    elif    1 <= h < 2: rgb = [X, chroma, 0]
    elif    2 <= h < 3: rgb = [0, chroma, X]
    elif    3 <= h < 4: rgb = [0, X, chroma]
    elif    4 <= h < 5: rgb = [X, 0, chroma]
    else:   rgb = [chroma, 0, X]

    # Match the lightness
    m = l - chroma/2
    return [int((i+m) * 255) for i in rgb]

In [138]:
df['sequential_rgb'] = df['sequential_hsl'].apply(lambda x: [hslToRGB(i) for i in x])
df[['sequential_rgb']]

,sequential_rgb
0,"[[50, 0, 0], [91, 0, 0], [132, 0, 0], [173, 0,..."
1,"[[50, 0, 0], [91, 0, 0], [132, 0, 0], [173, 0,..."
2,"[[25, 25, 25], [71, 71, 71], [117, 117, 117], ..."


In [139]:
df['diverging_rgb'] = df['diverging_hsl'].apply(lambda x: [hslToRGB(i) for i in x])
df[['diverging_rgb']]

,diverging_rgb
0,"[[255, 163, 255], [255, 122, 255], [255, 81, 2..."
1,"[[255, 163, 255], [255, 122, 255], [255, 81, 2..."
2,"[[234, 234, 234], [239, 239, 239], [244, 244, ..."


### RGB to Hex

In [140]:
def rgbToHex(rgb):
    hexcode = [hex(i)[2:] for i in rgb]

    # six digit format
    for i in range(len(hexcode)):
        if len(hexcode[i]) != 2:
            hexcode[i] *= 2 
    
    return ''.join(hexcode)

In [141]:
df['sequential_hex'] = df['sequential_rgb'].apply(lambda x: [rgbToHex(i) for i in x])
df[['sequential_hex']]

,sequential_hex
0,"[320000, 5b0000, 840000, ad0000, d60000, ff000..."
1,"[320000, 5b0000, 840000, ad0000, d60000, ff000..."
2,"[191919, 474747, 757575, a3a3a3, d1d1d1, fffff..."


In [142]:
df['diverging_hex'] = df['diverging_rgb'].apply(lambda x: [rgbToHex(i) for i in x])
df[['diverging_hex']]

,diverging_hex
0,"[ffa3ff, ff7aff, ff51ff, ff28ff, ff00ff, d600d..."
1,"[ffa3ff, ff7aff, ff51ff, ff28ff, ff00ff, d600d..."
2,"[eaeaea, efefef, f4f4f4, f9f9f9, ffffff, d1d1d..."


Preview all conversions.

In [143]:
df

,hex,rgb,hsl,sequential_hsl,compliment_hsl,diverging_hsl,sequential_rgb,diverging_rgb,sequential_hex,diverging_hex
0,FF0000,"(255, 0, 0)","(0, 100, 50)","[(0, 100, 10), (0, 100, 18), (0, 100, 26), (0,...","(180, 100, 50)","[(180, 100, 82), (180, 100, 74), (180, 100, 66...","[[50, 0, 0], [91, 0, 0], [132, 0, 0], [173, 0,...","[[255, 163, 255], [255, 122, 255], [255, 81, 2...","[320000, 5b0000, 840000, ad0000, d60000, ff000...","[ffa3ff, ff7aff, ff51ff, ff28ff, ff00ff, d600d..."
1,00ff00,"(0, 255, 0)","(120, 100, 50)","[(120, 100, 10), (120, 100, 18), (120, 100, 26...","(300, 100, 50)","[(300, 100, 82), (300, 100, 74), (300, 100, 66...","[[50, 0, 0], [91, 0, 0], [132, 0, 0], [173, 0,...","[[255, 163, 255], [255, 122, 255], [255, 81, 2...","[320000, 5b0000, 840000, ad0000, d60000, ff000...","[ffa3ff, ff7aff, ff51ff, ff28ff, ff00ff, d600d..."
2,ffffff,"(255, 255, 255)","(0, 0, 100)","[(0, 0, 10), (0, 0, 28), (0, 0, 46), (0, 0, 64...","(180, 0, 100)","[(180, 0, 92), (180, 0, 94), (180, 0, 96), (18...","[[25, 25, 25], [71, 71, 71], [117, 117, 117], ...","[[234, 234, 234], [239, 239, 239], [244, 244, ...","[191919, 474747, 757575, a3a3a3, d1d1d1, fffff...","[eaeaea, efefef, f4f4f4, f9f9f9, ffffff, d1d1d..."
